# Dog Breed Recognition Project

## Project Basics

#### Problem

Our goal is to identify dog breed from a photo of the dog.  
The project is taken from [Kaggle Dog Breed Identification Competition](https://www.kaggle.com/c/dog-breed-identification/data).  
The machine learning problem is **supervised learning > multiclass classification**.  
Our task is to build a neural network image classifier using TensorFlow and TensorFlow Hub.

#### Evaluation

The evaluation metric set for the competition is Multiclass Log Loss.  
Our target matrix contains N Dogs x M Breeds, true breed = 1, rest = 0.  
Our model predicts a probability matrix with the same dimensions.  
Multiclass Log Loss measures the error of model predictions (the lower the better).  
Muticlass Log Loss is applied in image classification, natural language processing, and recommendation systems.

#### Data Source

Data is acquired from [Kaggle Dog Breed Identification Competition](https://www.kaggle.com/c/dog-breed-identification/data).

#### Features / Data Dictionary

Our model analyzes image files (unstructured data) > deep learning / transfer learning.  
There are 120 unique dog breeds in the training set > multiclass classification with 120 classes.  
There are 10 222 images in the training set.  
There are 10 357 images in the test set.

## Importing Libraries

In [ ]:
### importing tensorflow
import tensorflow as tflow
print(tflow.__version__)
import keras

### checking gpu availability
print(tflow.config.list_physical_devices())

### importing tensorflow hub
import tensorflow_hub as tfhub
print(tfhub.__version__)

### importing sklearn tools
from sklearn.model_selection import train_test_split

### other imports
from typing import List, Any
from pathlib import Path
from datetime import datetime
import numpy
from pandas import read_csv, Series, DataFrame, concat, get_dummies
from matplotlib import pyplot
from IPython.display import Image

## Data Acquisition

In [ ]:
### unzipping project data
#!unzip "drive/MyDrive/Colab Data/dog-recognition.zip" -d "drive/MyDrive/Colab Data/"

In [ ]:
### importing labels
labels_df = read_csv(filepath_or_buffer="drive/MyDrive/ColabData/DogRecognition/labels.csv")

In [ ]:
### exploring labels: head
labels_df.head()

In [ ]:
### exploring labels: info
labels_df.info()

In [ ]:
### exploring labels: unique breeds
unique_breeds = labels_df["breed"].to_numpy()
unique_breeds = numpy.unique(ar=unique_breeds)
len(unique_breeds)

In [ ]:
### exploring labels: mean of images/breed
round(number=labels_df["breed"].value_counts().mean(), ndigits=3)

Google recommends at least 10 images per class.  
We have adequate data with ~85 images per class on average.

## Preparing Data

#### Creating Image Filepaths

In [ ]:
### counting number of images in train folder
image_list = [image for image in Path("drive/MyDrive/ColabData/DogRecognition/train").iterdir()]
len(image_list)

In [ ]:
### creating image filepaths from image ids
labels_df["imagepath"] = "drive/MyDrive/ColabData/DogRecognition/train/" + labels_df["id"] + ".jpg"

In [ ]:
### exploring imagepaths: head
labels_df.head()

In [ ]:
### exploring imagepaths: info
labels_df.info()

In [ ]:
### exploring imagepaths: checking validity of random imagepath
print(labels_df.loc[9000, "breed"])
print()
print(labels_df.loc[9000, "imagepath"])
print()
Image(filename=labels_df.loc[9000, "imagepath"])

#### Reducing and Splitting

In [ ]:
### dataframe inits
train_df = DataFrame()
valid_df = DataFrame()

In [ ]:
### creating train and valid datasets
for breed in unique_breeds:
  work_df = labels_df.loc[labels_df["breed"] == breed].copy(deep=True)
  work_df = work_df.sample(n=12, random_state=42, ignore_index=True)
  train_df = concat(objs=[train_df, work_df.loc[:9]], ignore_index=True, copy=True)
  valid_df = concat(objs=[valid_df, work_df.loc[10:]], ignore_index=True, copy=True)

In [ ]:
### shuffling train dataset
train_df = train_df.sample(n=train_df.index.size, random_state=42, ignore_index=True)

In [ ]:
### verifying dimensions of train and valid datasets
train_df.shape, valid_df.shape

#### Creating Tensors

All machine learning algorithms require data in numerical format.  
So the first task is to turn images and labels into tensors.  
A tensor is a numerical matrix with n-dimensions, like a numpy ndarray.

In [ ]:
### function creating image tensor
def imageTensor(pInput_df=DataFrame(), pImage_size=224):
  """
  Creates an image tensor from image filepaths.
  """
  tensor_list = list()
  for index,row in pInput_df.iterrows():
    print(index)
    image_tensor = tflow.io.read_file(filename=row["imagepath"])
    image_tensor = tflow.image.decode_jpeg(contents=image_tensor, channels=3)
    image_tensor = tflow.image.convert_image_dtype(image=image_tensor, dtype=tflow.float32)
    image_tensor = tflow.image.resize(images=image_tensor, size=[pImage_size, pImage_size])
    tensor_list.append(image_tensor)
  return tflow.stack(values=tensor_list, axis=0)

In [ ]:
### creating and saving train images tensor
train_images = imageTensor(pInput_df=train_df, pImage_size=224)
numpy.save(arr=train_images.numpy(), file="drive/MyDrive/ColabData/DogRecognition/train-images.npy")

In [ ]:
### creating and saving valid images tensor
valid_images = imageTensor(pInput_df=valid_df, pImage_size=224)
numpy.save(arr=valid_images.numpy(), file="drive/MyDrive/ColabData/DogRecognition/valid-images.npy")

In [ ]:
### reloading image tensors
train_images = tflow.constant(value=numpy.load(file="drive/MyDrive/ColabData/DogRecognition/train-images.npy"))
valid_images = tflow.constant(value=numpy.load(file="drive/MyDrive/ColabData/DogRecognition/valid-images.npy"))
train_images.shape, valid_images.shape

In [ ]:
### function creating label tensor
def labelTensor(pInput_df=DataFrame()):
  """
  Creates a label tensor from breed names.
  """
  tensor_list = list()
  for index,row in pInput_df.iterrows():
    print(index)
    label_array = numpy.zeros(shape=120, dtype="int8")
    label_array[numpy.where(unique_breeds == row["breed"])] = 1
    tensor_list.append(tflow.constant(value=label_array))
  return tflow.stack(values=tensor_list, axis=0)

In [ ]:
### creating train labels tensor
train_labels = labelTensor(pInput_df=train_df)
train_labels.shape

In [ ]:
### creating valid labels tensor
valid_labels = labelTensor(pInput_df=valid_df)
valid_labels.shape

#### Data Batches

GPUs have limited amount of memory.  
The entire training dataset may not fit into GPU memory.  
To resolve this, we split our datasets into batches of ~32 tensors.  
The neural network sees only one batch at a time.

In [ ]:
### function creating data batches
def dataBatches(pInput_tensors=tuple(), pBatch_size=32):
  """
  Creates data batches from input tensors.
  """
  ### creating dataset
  data_set = tflow.data.Dataset.from_tensor_slices(tensors=pInput_tensors)
  ### creating and returning data batches
  return data_set.batch(batch_size=pBatch_size)

In [ ]:
### creating train data batches
train_batches = dataBatches(pInput_tensors=(train_images,train_labels), pBatch_size=32)
train_batches.element_spec

In [ ]:
### creating valid data batches
valid_batches = dataBatches(pInput_tensors=(valid_images,valid_labels), pBatch_size=32)
valid_batches.element_spec

#### Visualizing Datasets

In [ ]:
### function visualizing a data batch
def visualizeBatch(pImages=numpy.array([]), pLabels=numpy.array([])):
  """
  Displays images and labels from a data batch.
  """
  pyplot.figure(figsize=(8,12))
  for index,image,label in zip(range(1, 33), pImages, pLabels):
    axis = pyplot.subplot(8, 4, index)
    pyplot.imshow(X=image)
    pyplot.title(label=unique_breeds[label.argmax()], fontsize=8)
    pyplot.axis("off")
  return

In [ ]:
### visualizing first batch of train bathes
images,labels = next(train_batches.as_numpy_iterator())
visualizeBatch(pImages=images, pLabels=labels)

In [ ]:
### visualizing first batch of valid bathes
images,labels = next(valid_batches.as_numpy_iterator())
visualizeBatch(pImages=images, pLabels=labels)

## Building and Training a Model

#### Defining the Model

In [ ]:
### setting base model features
INPUT_SHAPE = [None, 224, 224, 3] # batch, height, width, colorchannel
MODEL_URL = "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/130-224-classification/2"
OUTPUT_SHAPE = len(unique_breeds) # number of classes (breeds)
NUM_EPOCHS = 100

**Optimal model parameters:**
- for binary classification: sigmoid (activation), binary crossentropy (loss)  
- for multiclass classification: softmax (activation), categorical crossentropy (loss)

In [ ]:
### function defining the model
def defineModel(pInput_shape=list(), pModel_url=str(), pOutput_shape=int()):
  """
  Defines a deep learning neural network model.
  """
  ### setting model layers
  cnn_model = tflow.keras.Sequential([
      tfhub.KerasLayer(handle=pModel_url), # input layer
      tflow.keras.layers.Dense(units=pOutput_shape, activation="softmax")]) # output layer
  ### compiling model (training characteristics)
  cnn_model.compile(
      loss=tflow.keras.losses.CategoricalCrossentropy(), # basically an error function
      optimizer=tflow.keras.optimizers.Adam(), # optimizer algorithm
      metrics=["accuracy"])
  ### building model (providing input shape)
  cnn_model.build(pInput_shape) # shape of input dataset
  return cnn_model

In [ ]:
### defining and exploring the model
vision_model = defineModel(pInput_shape=INPUT_SHAPE, pModel_url=MODEL_URL, pOutput_shape=OUTPUT_SHAPE)
print(type(vision_model))
vision_model.summary()

#### Creating Callbacks

**Callbacks are event handler functions that are called at certain model training events.**

The TensorBoard() callback saves a training log that helps in monitoring the training process.

In [ ]:
### loading tensorboard notebook extension
%load_ext tensorboard

In [ ]:
### creating tensorboard callback
def callbackTensorboard():
  log_root = Path("drive/MyDrive/ColabData/DogRecognition/training_logs")
  log_folder = log_root.joinpath(datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tflow.keras.callbacks.TensorBoard(log_dir=log_folder)

The EarlyStopping() callback halts training when a chosen metric stops improving.  
It is one of the ways of preventing overfitting.

In [ ]:
### creating early stopping callback
early_stopping = tflow.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=8)

#### Training the Model

In [ ]:
### function creating trained model
def trainModel():
  """
  Defines and trains the deep learning neural network.
  """
  ### defining model
  cnn_model = defineModel(pInput_shape=INPUT_SHAPE, pModel_url=MODEL_URL, pOutput_shape=OUTPUT_SHAPE)
  ### tensorboard session init
  tensor_board = callbackTensorboard()
  ### training model
  cnn_model.fit(
      x=train_batches,
      epochs=NUM_EPOCHS,
      validation_data=valid_batches,
      validation_freq=1,
      callbacks=[tensor_board,early_stopping])
  return cnn_model

In [ ]:
### training model
trained_model = trainModel()

#### Model Evaluation with TensorBoard

In [ ]:
### loading tensorboard
%tensorboard --logdir drive/MyDrive/ColabData/DogRecognition/training_logs

## Visualizing Predictions

In [ ]:
### making predictions
valid_preds = trained_model.predict(x=valid_batches, verbose=True)
valid_preds.shape

In [ ]:
### function plotting image, true label, and top predictions
def plotPreds(aImages=numpy.ndarray([]), aLabels=numpy.ndarray([]), aPreds=numpy.ndarray([]), aIndex=int()):
  """
  Plots an image of a dog, its true breed, and the top predicted breeds for sample aIndex.
  """
  ### figure init
  if aIndex < 0: aIndex = 0
  if len(aPreds) <= aIndex: aIndex = len(aPreds) - 1
  pyplot.figure(figsize=(7,4.5))
  ### plotting image and true label
  image_plot = pyplot.subplot(1, 2, 1)
  image_plot.imshow(X=aImages[aIndex])
  image_plot.set_title(label=unique_breeds[numpy.argmax(a=aLabels[aIndex])], fontsize=10)
  image_plot.set_yticks(ticks=[])
  image_plot.set_xticks(ticks=[])
  ### plotting top 5 predictions
  top_indexes = numpy.argsort(a=aPreds[aIndex])[-5:][::-1]
  pred_plot = pyplot.subplot(1, 2, 2)
  pred_plot.bar(height=aPreds[aIndex][top_indexes], x=range(5))
  pred_plot.set_title(label="Top Five Predictions", fontsize=10)
  pred_plot.set_ylabel(ylabel="Confidence Levels", fontsize=10)
  pred_plot.set_xticks(ticks=range(5), labels=unique_breeds[top_indexes], rotation="vertical")
  ### layout and returning
  pyplot.tight_layout(h_pad=0.1)
  return

In [ ]:
plotPreds(aImages=valid_images, aLabels=valid_labels, aPreds=valid_preds, aIndex=44)

Challenge: Create a confusion matrix of true labels versus predictions.

## Saving and Loading Model

In [ ]:
### function saving trained model
def saveModel(aModel=keras.src.engine.sequential.Sequential(), aFile_name=str()):
  """
  DocString
  """
  models_root = Path("drive/MyDrive/ColabData/DogRecognition/models")
  model_dir = models_root.joinpath(datetime.now().strftime("%Y%M%D-%H%M%S"))
  model_path = model_dir.joinpath(f"{aFile_name}.h5")
  print(f"Saving model: {model_path.resolve()}")
  return

#### Reducing Data: Working Subset

In [ ]:
### splitting data working / rest
PERCENT_IMAGES = 0.1 #@param {type:"slider", min:0.1, max:1.0, step:0.1}
rest_features, work_features, rest_targets, work_targets = train_test_split(
    features_series,
    targets_df,
    test_size=PERCENT_IMAGES,
    random_state=42)